In [44]:
#Import Pandas
import pandas as pd
from sqlalchemy import create_engine

In [45]:
#Read CSV
df = pd.read_csv('data.csv', parse_dates=True)

In [46]:
#Filter Columns
df = df[['IM_INCIDENT_KEY', 'INCIDENT_DATE_TIME', 'ARRIVAL_DATE_TIME', 'TOTAL_INCIDENT_DURATION', 'ACTION_TAKEN1_DESC', 'ACTION_TAKEN2_DESC', 'ZIP_CODE', 'BOROUGH_DESC']]

In [47]:
#Get All columns name
#df.columns

In [48]:
#Check memory Usage (6.1 MB)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119324 entries, 0 to 119323
Data columns (total 8 columns):
IM_INCIDENT_KEY            119324 non-null int64
INCIDENT_DATE_TIME         119324 non-null object
ARRIVAL_DATE_TIME          118636 non-null object
TOTAL_INCIDENT_DURATION    119324 non-null int64
ACTION_TAKEN1_DESC         119324 non-null object
ACTION_TAKEN2_DESC         115132 non-null object
ZIP_CODE                   119322 non-null float64
BOROUGH_DESC               119324 non-null object
dtypes: float64(1), int64(2), object(5)
memory usage: 7.3+ MB


In [49]:
#Reduce memory Usage (4.9 MB)
df['ACTION_TAKEN1_DESC'] = df['ACTION_TAKEN1_DESC'].astype('category')
df['ACTION_TAKEN2_DESC'] = df['ACTION_TAKEN2_DESC'].astype('category')
df['BOROUGH_DESC'] = df['BOROUGH_DESC'].astype('category')


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119324 entries, 0 to 119323
Data columns (total 8 columns):
IM_INCIDENT_KEY            119324 non-null int64
INCIDENT_DATE_TIME         119324 non-null object
ARRIVAL_DATE_TIME          118636 non-null object
TOTAL_INCIDENT_DURATION    119324 non-null int64
ACTION_TAKEN1_DESC         119324 non-null category
ACTION_TAKEN2_DESC         115132 non-null category
ZIP_CODE                   119322 non-null float64
BOROUGH_DESC               119324 non-null category
dtypes: category(3), float64(1), int64(2), object(2)
memory usage: 4.9+ MB


In [50]:
# Drop NA Valus
df = df.dropna()
df.head(3)

,IM_INCIDENT_KEY,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC
2,63476614,06/30/2018 11:22:51 PM,06/30/2018 11:27:18 PM,995,86 - Investigate,45 - Remove hazard,10472.0,2 - Bronx
3,63476562,06/30/2018 11:05:55 PM,06/30/2018 11:11:39 PM,671,44 - Hazardous materials leak control & contai...,64 - Shut down system,10452.0,2 - Bronx
4,63476510,06/30/2018 10:53:15 PM,06/30/2018 10:58:49 PM,1384,44 - Hazardous materials leak control & contai...,64 - Shut down system,10463.0,2 - Bronx


In [51]:
# Get Columns Types
df.dtypes

IM_INCIDENT_KEY               int64
INCIDENT_DATE_TIME           object
ARRIVAL_DATE_TIME            object
TOTAL_INCIDENT_DURATION       int64
ACTION_TAKEN1_DESC         category
ACTION_TAKEN2_DESC         category
ZIP_CODE                    float64
BOROUGH_DESC               category
dtype: object

In [52]:
# Split Date/Time and export to a separate dataframe
INCIDENT_DATE_TIME = df["INCIDENT_DATE_TIME"].str.split(" ", n = 1, expand = True) 
ARRIVAL_DATE_TIME = df["ARRIVAL_DATE_TIME"].str.split(" ", n = 1, expand = True) 

In [53]:
#Delete column in the original DF with the date and time combined (we will merge the new df)
del df['INCIDENT_DATE_TIME']
del df['ARRIVAL_DATE_TIME']

In [54]:
#Reset Indexes (they now start at two)

#First DF
df.reset_index(inplace=True, drop=True)

#Second DF
INCIDENT_DATE_TIME.reset_index(inplace=True, drop=True)

#Third DF
ARRIVAL_DATE_TIME.reset_index(inplace=True, drop=True)

In [55]:
#Rename new DF columns names

#Incident Date/time
INCIDENT_DATE_TIME.rename(columns={0: 'Incident Date', 1: 'Incident Time'}, inplace=True)

#Arrival Date/Time
ARRIVAL_DATE_TIME.rename(columns={0: 'Arrival Date', 1: 'Arrival Time'}, inplace=True)

In [56]:
# Join the two DFs
df = df.join(INCIDENT_DATE_TIME, how='outer')
df = df.join(ARRIVAL_DATE_TIME, how='outer')

In [57]:
df.head(3)

,IM_INCIDENT_KEY,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC,Incident Date,Incident Time,Arrival Date,Arrival Time
0,63476614,995,86 - Investigate,45 - Remove hazard,10472.0,2 - Bronx,06/30/2018,11:22:51 PM,06/30/2018,11:27:18 PM
1,63476562,671,44 - Hazardous materials leak control & contai...,64 - Shut down system,10452.0,2 - Bronx,06/30/2018,11:05:55 PM,06/30/2018,11:11:39 PM
2,63476510,1384,44 - Hazardous materials leak control & contai...,64 - Shut down system,10463.0,2 - Bronx,06/30/2018,10:53:15 PM,06/30/2018,10:58:49 PM


In [58]:
#Import weather data frame and parse dates
df_weather = pd.read_csv("nycweatherdata.csv", parse_dates=True)
df_weather.head(3)


,STATION,DATE,TMAX,TMIN
0,USW00094728,1/1/13,40,26
1,USW00094728,1/2/13,33,22
2,USW00094728,1/3/13,32,24


In [59]:
# Rename column name/ we need to match the two columns name in the weather and our original df to join on
df_weather.rename(columns={'DATE': 'Incident Date'}, inplace=True)
df_weather.head(3)

,STATION,Incident Date,TMAX,TMIN
0,USW00094728,1/1/13,40,26
1,USW00094728,1/2/13,33,22
2,USW00094728,1/3/13,32,24


In [61]:
# Convert the dates columns to date_time object to join on
df['Incident Date'] = pd.to_datetime(df['Incident Date'])
df_weather['Incident Date'] = pd.to_datetime(df_weather['Incident Date'])

In [62]:
# Join weather and our DF
final_df = df_weather.merge(df, on='Incident Date')

In [63]:
# Delete Station column (we don't need it)
del final_df['STATION']

In [64]:
# verify join worked fine
final_df.head(3)

,Incident Date,TMAX,TMIN,IM_INCIDENT_KEY,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC,Incident Time,Arrival Date,Arrival Time
0,2013-01-01,40,26,55675645,1108,44 - Hazardous materials leak control & contai...,64 - Shut down system,10455.0,2 - Bronx,11:30:10 PM,01/01/2013,11:34:39 PM
1,2013-01-01,40,26,55675621,461,44 - Hazardous materials leak control & contai...,64 - Shut down system,10027.0,1 - Manhattan,11:14:24 PM,01/01/2013,11:19:08 PM
2,2013-01-01,40,26,55675611,829,44 - Hazardous materials leak control & contai...,64 - Shut down system,11207.0,4 - Brooklyn,11:08:08 PM,01/01/2013,11:10:30 PM


In [65]:
# Get rid of the .0 that is in the zip code column
final_df['ZIP_CODE'] = final_df['ZIP_CODE'].astype(int)

In [66]:
# Check zipcode formatting
final_df.head(3)

,Incident Date,TMAX,TMIN,IM_INCIDENT_KEY,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC,Incident Time,Arrival Date,Arrival Time
0,2013-01-01,40,26,55675645,1108,44 - Hazardous materials leak control & contai...,64 - Shut down system,10455,2 - Bronx,11:30:10 PM,01/01/2013,11:34:39 PM
1,2013-01-01,40,26,55675621,461,44 - Hazardous materials leak control & contai...,64 - Shut down system,10027,1 - Manhattan,11:14:24 PM,01/01/2013,11:19:08 PM
2,2013-01-01,40,26,55675611,829,44 - Hazardous materials leak control & contai...,64 - Shut down system,11207,4 - Brooklyn,11:08:08 PM,01/01/2013,11:10:30 PM


In [67]:
#Remove numbers that are nex to the Bourough (Maps need only the name with no nums)
final_df['BOROUGH_DESC'] = final_df['BOROUGH_DESC'].map(lambda x: str(x)[4:])

In [68]:
final_df.head(3)

,Incident Date,TMAX,TMIN,IM_INCIDENT_KEY,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC,Incident Time,Arrival Date,Arrival Time
0,2013-01-01,40,26,55675645,1108,44 - Hazardous materials leak control & contai...,64 - Shut down system,10455,Bronx,11:30:10 PM,01/01/2013,11:34:39 PM
1,2013-01-01,40,26,55675621,461,44 - Hazardous materials leak control & contai...,64 - Shut down system,10027,Manhattan,11:14:24 PM,01/01/2013,11:19:08 PM
2,2013-01-01,40,26,55675611,829,44 - Hazardous materials leak control & contai...,64 - Shut down system,11207,Brooklyn,11:08:08 PM,01/01/2013,11:10:30 PM


In [69]:
final_df = final_df.rename(columns = {"Incident Date": "Date",
                                 "Arrival Date":"Arrival_Date",
                                 "Arrival Time": "Arrival_Time",
                                    "Incident Time":"Incident_Time"})

In [70]:
final_df.head()

,Date,TMAX,TMIN,IM_INCIDENT_KEY,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC,Incident_Time,Arrival_Date,Arrival_Time
0,2013-01-01,40,26,55675645,1108,44 - Hazardous materials leak control & contai...,64 - Shut down system,10455,Bronx,11:30:10 PM,01/01/2013,11:34:39 PM
1,2013-01-01,40,26,55675621,461,44 - Hazardous materials leak control & contai...,64 - Shut down system,10027,Manhattan,11:14:24 PM,01/01/2013,11:19:08 PM
2,2013-01-01,40,26,55675611,829,44 - Hazardous materials leak control & contai...,64 - Shut down system,11207,Brooklyn,11:08:08 PM,01/01/2013,11:10:30 PM
3,2013-01-01,40,26,55675547,1025,44 - Hazardous materials leak control & contai...,64 - Shut down system,11373,Queens,10:26:05 PM,01/01/2013,10:29:29 PM
4,2013-01-01,40,26,55675480,1054,44 - Hazardous materials leak control & contai...,64 - Shut down system,11360,Queens,09:33:56 PM,01/01/2013,09:39:20 PM


In [71]:
engine = create_engine('sqlite:///coned.db', echo=True)

In [72]:
final_df.to_sql('ConEdDB', con=engine, if_exists='replace')

2019-08-24 12:16:06,663 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-08-24 12:16:06,664 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 12:16:06,664 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-08-24 12:16:06,665 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 12:16:06,666 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ConEdDB")
2019-08-24 12:16:06,666 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 12:16:06,668 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ConEdDB")
2019-08-24 12:16:06,668 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 12:16:06,669 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-08-24 12:16:06,670 INFO sqlalchemy.engine.base.Engine ()
2019-08-24 12:16:06,671 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ConEdDB")
2019-08-24 12:16:06,671 INFO sqlalchemy.engine.base.Engine ()
201

In [73]:
year_data = final_df[final_df['Date'].dt.year == 2013]
year_data

,Date,TMAX,TMIN,IM_INCIDENT_KEY,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC,Incident_Time,Arrival_Date,Arrival_Time
0,2013-01-01,40,26,55675645,1108,44 - Hazardous materials leak control & contai...,64 - Shut down system,10455,Bronx,11:30:10 PM,01/01/2013,11:34:39 PM
1,2013-01-01,40,26,55675621,461,44 - Hazardous materials leak control & contai...,64 - Shut down system,10027,Manhattan,11:14:24 PM,01/01/2013,11:19:08 PM
2,2013-01-01,40,26,55675611,829,44 - Hazardous materials leak control & contai...,64 - Shut down system,11207,Brooklyn,11:08:08 PM,01/01/2013,11:10:30 PM
3,2013-01-01,40,26,55675547,1025,44 - Hazardous materials leak control & contai...,64 - Shut down system,11373,Queens,10:26:05 PM,01/01/2013,10:29:29 PM
4,2013-01-01,40,26,55675480,1054,44 - Hazardous materials leak control & contai...,64 - Shut down system,11360,Queens,09:33:56 PM,01/01/2013,09:39:20 PM
5,2013-01-01,40,26,55675481,1515,44 - Hazardous materials leak control & contai...,64 - Shut down system,10314,Staten Island,09:33:49 PM,01/01/2013,09:38:23 PM
6,2013-01-01,40,26,55675429,2707,44 - Hazardous materials leak control & contai...,64 - Shut down system,10019,Manhattan,09:10:36 PM,01/01/2013,09:14:47 PM
7,2013-01-01,40,26,55675384,899,44 - Hazardous materials leak control & contai...,64 - Shut down system,11221,Brooklyn,08:40:27 PM,01/01/2013,08:42:33 PM
8,2013-01-01,40,26,55675098,961,44 - Hazardous materials leak control & contai...,64 - Shut down system,10456,Bronx,06:50:03 PM,01/01/2013,06:54:08 PM
9,2013-01-01,40,26,55675057,1610,44 - Hazardous materials leak control & contai...,64 - Shut down system,10035,Manhattan,06:30:09 PM,01/01/2013,06:33:29 PM


In [74]:
year_data.Date.values.tolist()

[1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1356998400000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 1357084800000000000,
 135708480

In [75]:
final_df.to_csv("final_leaks.csv")

In [76]:
year_2013 = year_data[year_data['Date'].dt.year == 2013]

In [77]:
grouped = pd.DataFrame(year_2013.groupby('ZIP_CODE').sum()['TMAX'])
grouped.rename(columns = {'TMAX': 'Total Leaks'}, inplace=True)
grouped.to_dict()

{'Total Leaks': {10001: 2376,
  10002: 11473,
  10003: 6140,
  10004: 651,
  10005: 38,
  10007: 992,
  10009: 10560,
  10010: 1993,
  10011: 3617,
  10012: 1668,
  10013: 3776,
  10014: 4310,
  10016: 3781,
  10017: 1566,
  10018: 1006,
  10019: 3612,
  10020: 69,
  10021: 5789,
  10022: 2521,
  10023: 3399,
  10024: 5977,
  10025: 8780,
  10026: 6365,
  10027: 12602,
  10028: 3029,
  10029: 15338,
  10030: 5702,
  10031: 7454,
  10032: 8142,
  10033: 3542,
  10034: 3835,
  10035: 9514,
  10036: 2562,
  10037: 3604,
  10038: 2588,
  10039: 5829,
  10040: 5320,
  10128: 4017,
  10280: 144,
  10301: 4702,
  10302: 1346,
  10303: 2380,
  10304: 4247,
  10305: 4409,
  10306: 5932,
  10307: 1019,
  10308: 1772,
  10309: 2062,
  10310: 2645,
  10312: 3985,
  10314: 5991,
  10451: 10677,
  10452: 6456,
  10453: 9817,
  10454: 9379,
  10455: 5100,
  10456: 16548,
  10457: 8623,
  10458: 8651,
  10459: 5314,
  10460: 7217,
  10461: 5586,
  10462: 6767,
  10463: 9437,
  10464: 819,
  10465: 628

In [78]:
month_data = year_data[year_data['Date'].dt.month == 1]
month_data = month_data[['Date', 'TMAX']]

In [79]:
month_data.groupby('Date')['TMAX'].max()
grouped_month = pd.DataFrame(month_data.groupby('Date')['TMAX'].max())
grouped_month.rename(columns = {'TMAX': 'Temp'}, inplace=True)
grouped_month.to_dict()

{'Temp': {Timestamp('2013-01-01 00:00:00'): 40,
  Timestamp('2013-01-02 00:00:00'): 33,
  Timestamp('2013-01-03 00:00:00'): 32,
  Timestamp('2013-01-04 00:00:00'): 37,
  Timestamp('2013-01-05 00:00:00'): 42,
  Timestamp('2013-01-06 00:00:00'): 46,
  Timestamp('2013-01-07 00:00:00'): 45,
  Timestamp('2013-01-08 00:00:00'): 48,
  Timestamp('2013-01-09 00:00:00'): 49,
  Timestamp('2013-01-10 00:00:00'): 47,
  Timestamp('2013-01-11 00:00:00'): 46,
  Timestamp('2013-01-12 00:00:00'): 47,
  Timestamp('2013-01-13 00:00:00'): 50,
  Timestamp('2013-01-14 00:00:00'): 56,
  Timestamp('2013-01-15 00:00:00'): 38,
  Timestamp('2013-01-16 00:00:00'): 37,
  Timestamp('2013-01-17 00:00:00'): 43,
  Timestamp('2013-01-18 00:00:00'): 35,
  Timestamp('2013-01-19 00:00:00'): 51,
  Timestamp('2013-01-20 00:00:00'): 53,
  Timestamp('2013-01-21 00:00:00'): 32,
  Timestamp('2013-01-22 00:00:00'): 27,
  Timestamp('2013-01-23 00:00:00'): 20,
  Timestamp('2013-01-24 00:00:00'): 22,
  Timestamp('2013-01-25 00:00:00

In [80]:
day_leak_count = month_data.groupby([month_data['Date'].dt.day]).count()

In [81]:
day_leak_count

,Date,TMAX
Date,,
1,35,35
2,46,46
3,44,44
4,33,33
5,53,53
6,42,42
7,55,55
8,40,40
9,35,35


In [82]:
day_leak_count.Date.values


array([35, 46, 44, 33, 53, 42, 55, 40, 35, 40, 50, 34, 33, 48, 43, 40, 40,
       30, 45, 30, 40, 35, 43, 60, 51, 30, 40, 39, 38, 48, 27])

In [83]:
day_temp_values = month_data.groupby([month_data['Date'].dt.day])['TMAX'].max()

In [84]:
day_temp_values.values

array([40, 33, 32, 37, 42, 46, 45, 48, 49, 47, 46, 47, 50, 56, 38, 37, 43,
       35, 51, 53, 32, 27, 20, 22, 24, 27, 34, 36, 49, 59, 61])

In [85]:
print(len(day_leak_count))

31


In [86]:
data = {
    "Day" : day_leak_count.index,
    "# of Leaks": day_leak_count.Date.values,
    "Dail Max Temp" : day_temp_values.values
}


In [87]:
data

{'Day': Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
             18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
            dtype='int64', name='Date'),
 '# of Leaks': array([35, 46, 44, 33, 53, 42, 55, 40, 35, 40, 50, 34, 33, 48, 43, 40, 40,
        30, 45, 30, 40, 35, 43, 60, 51, 30, 40, 39, 38, 48, 27]),
 'Dail Max Temp': array([40, 33, 32, 37, 42, 46, 45, 48, 49, 47, 46, 47, 50, 56, 38, 37, 43,
        35, 51, 53, 32, 27, 20, 22, 24, 27, 34, 36, 49, 59, 61])}